In [189]:
# load config

import json

with open('config.json') as f:
    config = json.load(f)

In [190]:
# init db

from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base

engine = create_engine(config['db'])
# 用 automap_base 自动反射成库中已有的类型
# https://docs.sqlalchemy.org/en/14/orm/extensions/automap.html
Base = automap_base()
Base.prepare(engine, reflect=True)

print(list(Base.classes))

[<class 'sqlalchemy.ext.automap.artists'>, <class 'sqlalchemy.ext.automap.albums'>, <class 'sqlalchemy.ext.automap.comments'>, <class 'sqlalchemy.ext.automap.users'>, <class 'sqlalchemy.ext.automap.tracks'>, <class 'sqlalchemy.ext.automap.playlists'>, <class 'sqlalchemy.ext.automap.playlist_tracks'>, <class 'sqlalchemy.ext.automap.track_emotions'>]


In [191]:
# 注意这里反射出的类名是带 s 的，直接是表名。

# 还有 playlist_tracks 是自定义的，automap 不出来 many2many 关系。
# (See https://docs.sqlalchemy.org/en/14/orm/extensions/automap.html#many-to-many-relationships)
# 但这里好像不用 playlist，就先不管了。

Track = Base.classes.tracks
Comment = Base.classes.comments

In [192]:
vars(Track)

mappingproxy({'__table__': Table('tracks', MetaData(), Column('id', BIGINT(), table=<tracks>, primary_key=True, nullable=False, server_default=DefaultClause(<sqlalchemy.sql.elements.TextClause object at 0x1164586a0>, for_update=False)), Column('name', TEXT(), table=<tracks>), Column('pop', BIGINT(), table=<tracks>), Column('publish_time', BIGINT(), table=<tracks>), Column('lyric', TEXT(), table=<tracks>), schema=None),
              '__module__': 'sqlalchemy.ext.automap',
              '__doc__': None,
              '_sa_class_manager': <ClassManager of <class 'sqlalchemy.ext.automap.tracks'> at 11f268220>,
              '__init__': <function sqlalchemy.orm.instrumentation.__init__(self, **kwargs)>,
              'comments_collection': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x11f42b590>,
              'albums_collection': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x11f42b680>,
              'id': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x11f42b450>,
 

In [193]:
# 试试查询

from sqlalchemy.orm import Session

session = Session(engine)

tks = session.query(Track).order_by(Track.pop.desc())
for t in tks[0: 3]:
    artists = list(map(lambda x: x.name, t.artists_collection))
    comments = list(map(lambda x: x.content, t.comments_collection))
    print(f'{t.id=}, {t.name=}, {t.pop=}\n\t{artists=}\n\t{comments=}')

t.id=557583289, t.name='Honey', t.pop=100
	artists=['Raveena']
	comments=['raveena - Honey 2018-5-9 单曲已上传 手机处理中 正在吃饭 ', '还没分别，已在心里写信', '你这样子我会胡思乱想的', '“今晚的星星和月亮都像你 遥不可及地好看”', '““躺在蜂蜜罐子里泡个澡\n再去面包的奶油层里打个滚儿\n做个好吃的梦”\n\n/魏什么是泡芙呀 \u200b\u200b\u200b”----德卡先生的信箱', '温的水，柔的人，交织的温存。', '好啦，我错了，改改就好啦。', '想让你浑身乏力时喊我的名字', '狗在家中坐  粮从天上来', '狗在被窝躺 粮从天上来', '这世上至少有一半冷漠都是害羞', '想要一个拥抱 用体温把刚出炉的委屈烘烤掉', '可是那样太容易错过人了[流泪]', "最后五个 You're honey 不如翻译成\n你是蜂蜜 你好甜蜜 你是我的爱人 你是我的唯一 我好爱你\nWaterfall falling through my roof. 爱意如瀑布洒落房间\nMy cup is fuller when you're here.你在的时候 爱意满溢 满杯甜蜜。\n来自一个英语专业同学的小声逼逼💜🌟✨愿喜💜", '那个和我品味一样奇怪的混蛋男人什么时候才出现', '瀑布的这句太棒啦！！', '牛奶和蜂蜜一起喝会窜稀的哦', '遇见一个喜欢爵士的女孩，推荐的歌像清风摇曳。', '我始终觉得，爱意就是要让对方感受到。哪怕被回绝、被辜负也总好过堆在心里腐烂掉。要学会大大方方地表达爱意，爱不是冷冰冰或者讲反话。爱应该是炽热的，永远都是。', '有喜欢这首歌的女生请联系我，我想和你原地结婚']
t.id=565841054, t.name='Say My Name', t.pop=100
	artists=['NIKI']
	comments=['撩的都是不喜欢的，真正喜欢的都是小心翼翼的', '每当天色暗下来，想念就渐渐发芽，长出你的名字来。', '也许我们的温柔也就那夜晚的几个小时[心碎] 不会再见', 'Niki好像是被A妹力捧的新声代耶 难怪嗓音独特中带着杏菜的调调', '有一说一这歌我不敢轻易分享🌚', '来北京后的第一件事就是找公寓\

In [194]:
# new table
# 只是建个表，只需要执行一次。
# 这好像也不比手写 SQL 快耶。。。下次别这么搞了。

'''
from sqlalchemy import Column, Text, Float, ForeignKey, BigInteger
from sqlalchemy.orm import relationship


class TrackEmotion(Base):
    __tablename__ = 'track_emotions'

    track_id = Column(BigInteger, ForeignKey('tracks.id'), primary_key=True)
    emotion = Column(Text, primary_key=True)
    intensity = Column(Float)

    track = relationship('Track')

Base.metadata.create_all(engine)
# Track.track_emotions = relationship("TrackEmotion")

# Base.metadata.create_all(engine)
'''
TrackEmotion = Base.classes.track_emotions

In [195]:
tks = session.query(Track)
for t in tks[0: 3]:
    comments = t.comments_collection
    liked_counts = [c.liked_count for c in comments]
    sum_liked = sum(liked_counts)
    print(sum_liked)
    liked_weights = [liked / sum_liked for liked in liked_counts]
    for comment, liked_weight in zip(comments, liked_weights):
        print(comment.liked_count, liked_weight, liked_weight * sum_liked)

0
0
144797
33495 0.23132385339475264 33495.0
13371 0.09234307340621697 13371.0
13741 0.09489837496633217 13741.0
3375 0.02330849396051023 3374.9999999999995
2867 0.019800133980676395 2867.0
2293 0.01583596345228147 2293.0
12673 0.08752253154416183 12673.0
6041 0.04172047763420513 6041.000000000001
3782 0.026119325676636947 3782.0
7114 0.0491308521585392 7114.0
2420 0.016713053447239928 2420.0
1793 0.012382853235909584 1793.0
2404 0.016602553920316027 2404.0
2272 0.01569093282319385 2272.0
5951 0.04109891779525819 5951.0
12520 0.08646587981795203 12520.0
1734 0.0119753862303777 1734.0
9807 0.06772930378391817 9807.0
7144 0.04933803877152151 7144.0


In [200]:
from math import log
import emotext

Emo = emotext.Emotions()


def track_emotion(t: Track):
    lyrics = t.lyric
    comments = t.comments_collection
    if (not comments) and (not lyrics):
        return

    texts = [c.content for c in comments]
    liked_counts = [c.liked_count for c in comments]
    if lyrics:
        texts.append(lyrics)
        liked_counts.append(sum(liked_counts) * 10)

    liked_weights = [log(1.01 + l, 10) for l in liked_counts]

    track_emotions = emotext.EmotionCountResult().emotions

    for text, weight in zip(texts, liked_weights):
        emo_result = Emo.emotion_count(text)

        # print(text, weight, sorted(emo_result.emotions.items(), key=lambda x: -x[-1]))
        for k, v in emo_result.emotions.items():
            if not v:
                continue
            track_emotions[k] += v * weight

    return track_emotions


def add_emotion(t: Track, verbose=False):
    emo_res = track_emotion(t)
    if not emo_res:
        return

    sum_v = sum(emo_res.values())

    if sum_v:
        t.track_emotions_collection = []

    for e, v in emo_res.items():
        if not v:
            continue
        v = v / sum_v
        t.track_emotions_collection.append(TrackEmotion(emotion=e, intensity=v))
    session.commit()

    if verbose:
        print(t.name, [k for k, v in sorted(emo_res.items(), key=lambda x: -x[-1]) if v])

tks = session.query(Track)
for t in tks[0: 3]:
    add_emotion(t, verbose=True)

Boo Thang ['PB', 'NN', 'NB', 'PE']
Gimme That (Prod. Stally, Colde) ['PB', 'NN', 'PH']
Say My Name ['PB', 'PH', 'NB', 'NC', 'PG', 'PA', 'PF', 'NN', 'PE', 'ND', 'NE']
